In [6]:
!git clone https://github.com/nishantkushwaha-1999/AlphaZero_C4.git
!pip install torchinfo
!pip install pygame

Cloning into 'AlphaZero_C4'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 7), reused 25 (delta 4), pack-reused 0
Receiving objects: 100% (33/33), 39.27 KiB | 3.57 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [3]:
import os
path = os.getcwd() + "/AlphaZero_C4"
os.chdir(path)
print(path)

In [ ]:
from google.colab import drive
drive.mount('./drive', force_remount=True)

In [1]:
import numpy as np
import torch
from Game.C4 import Game_C4
from AlphaZero.model import ResNet_C4
from AlphaZero.AlphaZero import AlphaZero
from torchinfo import summary

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.10.11)


# Train AlphaZero_C4

### Train from Scratch

In [4]:
game_c4 = Game_C4()

args = {
    'board_dim': (game_c4.rows, game_c4.columns),
    'n_actions': game_c4.action_size,
    'n_res_blocks': 10,
    'n_hidden': 64,
    'C': 3,
    'lr': 0.01,
    'weight_decay': 0.001,
    'num_searches': 800,
    'n_iters': 50,
    'n_selfPlay': 150,
    'n_parallel': True,
    'n_parallel_games': 50,
    'epochs': 4,
    'batch_size': 128,
    'temperature': 1.2,
    'dirichlet_epsilon': 0.25,
    'dirichlet_alpha': 0.65
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet_C4(
    board_dim=args['board_dim'],
    n_actions=args['n_actions'],
    n_res=args['n_res_blocks'],
    n_hidden=args['n_hidden'],
    device=device
)
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
architecture = summary(model, input_size=(args['batch_size'], 3, game_c4.rows, game_c4.columns), verbose=0)
print(architecture)

Layer (type:depth-idx)                   Output Shape              Param #
ResNet_C4                                [10, 1]                   --
├─Sequential: 1-1                        [10, 64, 6, 7]            --
│    └─Conv2d: 2-1                       [10, 64, 6, 7]            1,792
│    └─BatchNorm2d: 2-2                  [10, 64, 6, 7]            128
│    └─ReLU: 2-3                         [10, 64, 6, 7]            --
├─ModuleList: 1-2                        --                        --
│    └─ResBlock: 2-4                     [10, 64, 6, 7]            --
│    │    └─Conv2d: 3-1                  [10, 64, 6, 7]            36,928
│    │    └─BatchNorm2d: 3-2             [10, 64, 6, 7]            128
│    │    └─Conv2d: 3-3                  [10, 64, 6, 7]            36,928
│    │    └─BatchNorm2d: 3-4             [10, 64, 6, 7]            128
│    └─ResBlock: 2-5                     [10, 64, 6, 7]            --
│    │    └─Conv2d: 3-5                  [10, 64, 6, 7]            36,9

In [5]:
alphazero = AlphaZero()
alphazero.learn(model, optimizer, game_c4, args, 
                save_path="/content/AlphaZero_C4/drive/MyDrive/AlphaZero_C4/saved_models", 
                n_parallel=args['n_parallel'])

Initializing...
Iter 1 of 50
Self Play: Playing 50 games parallely


2100 fills remaining: :  33%|███▎      | 1/3 [13:51<27:43, 831.75s/it]


KeyboardInterrupt: 

### Load & Train

In [2]:
game_c4 = Game_C4()

alphazero = AlphaZero()
alphazero.load_and_resume(game_c4, "/Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/", train=True)

Resuming Training:
Iter 3 of 4
Self Play: Playing 2 games parallely


27 fills remaining: : 100%|██████████| 2/2 [00:38<00:00, 19.24s/it]


Model Train:


Epoch 1/2: 100%|██████████| 30/30 [00:00<00:00, 199.02it/s]


Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/****_3_Connect_4.pt
Iter 4 of 4
Self Play: Playing 2 games parallely


32 fills remaining: : 100%|██████████| 2/2 [00:39<00:00, 19.74s/it]


Model Train:


Epoch 1/2: 100%|██████████| 30/30 [00:00<00:00, 179.49it/s]


Model saved at: /Volumes/Storage/Git Repos/AlphaZero_C4/saved_models/****_4_Connect_4.pt
